In [3]:
import numpy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder
import cv2


torch.cuda.empty_cache()

In [7]:
cap = cv2.VideoCapture('http://192.168.0.107:8080/video')

while True:
    status, photo = cap.read()
    cv2.imshow('hi', photo)
    if cv2.waitKey(1) == 13:
        break
cv2.destroyAllWindows()
cap.release()

In [12]:
import torch
from torchvision.transforms import transforms
from PIL import Image
from pathlib import Path


# # To get the weights from the .pth file!
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.cnn = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.LeakyReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=2)
        self.relu2 = nn.LeakyReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(8*65*65, 1000)
        self.fc2 = nn.Linear(1000, 8)
        
        
    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model = NeuralNetwork()
checkpoint = torch.load(Path('tomato.pk1'))
model.load_state_dict(checkpoint)

# # Preprocess any image before passing it to the model
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(256),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomInvert(),
    transforms.RandomRotation(30),
])


image = Image.open(Path('yellow.jpg'))

input = test_transform(image)

input = input.view(1, 3, 256,256)

output = model(input)
print(output)

prediction = int(torch.max(output.data, 1)[1].numpy())
print(prediction)

if (prediction == 0):
    print ('Bacterial_spot')
if (prediction == 1):
    print ('Early_blight')
if (prediction == 2):
    print ('Late_blight')
if (prediction == 3):
    print ('Septoria_leaf_spot')
if (prediction == 4):
    print ('Two-spotted_spider_mite')
if (prediction == 5):
    print ('Target_Spot')
if (prediction == 6):
    print ('Tomato_Yellow_Leaf_Curl_Virus')
if (prediction == 7):
    print ('Tomato___healthy')

tensor([[ -12.5842,   -3.3236,   39.2880,  -22.7289,    5.7179,   39.4954,
           14.9348, -114.3501]], grad_fn=<AddmmBackward0>)
5
Target_Spot
